Right now, I've moved some of the constructors to be their own types. I think our weights will be clearer & more consistent if:

- W is the most generic type of weights object, and reflects arbitrarily-constructed weights from at least one `neighbor` dictionary.
- Rook, Queen, DistanceBand & Kernel, etc, reflect basic types of either `neighbor` construction or `weights` construction. 
- We can make flyweight classes (like AdaptiveKernel) that just call their parent (Kernel) with special options, and that's fine. 

In [1]:
import pysal as ps

I've been doing this cleanup & new work in parallel to `weights`, over in `weights2`. 

In [2]:
from pysal.weights2 import contiguity as cont, distance as dist
from pysal.weights2.weights import W as W2

In [3]:
path = ps.examples.get_path('south.shp')
psframe = ps.pdio.read_files(path)
filehandler = ps.open(path)

In [4]:
try:
    import shapely.geometry as geom
    sh_iterable = [geom.asShape(sh) for sh in psframe.geometry]
except ImportError:
    from warnings import warn as Warn
    Warn('No shapely :(')

I'll be showing Rook, but Queen has this, and I'm working on them for Distance methods. 

In [5]:
R1 = cont.Rook.from_iterable(sh_iterable)
R2 = cont.Rook.from_dataframe(psframe)
R3 = cont.Rook.from_shapefile(path)
Rreference = ps.rook_from_shapefile(path)

In [6]:
R1.neighbors == R2.neighbors == R3.neighbors == Rreference.neighbors

True

In [7]:
Q1 = cont.Queen.from_iterable(sh_iterable)
Q2 = cont.Queen.from_dataframe(psframe)
Q3 = cont.Queen.from_shapefile(path)
Qreference = ps.queen_from_shapefile(path)

In [8]:
Q1.neighbors == Q2.neighbors == Q3.neighbors == Qreference.neighbors

True

The only difference in the weights algorithms is in using ContiguityWeightsPolygons vs. ContiguityWeights_binning. Under the hood, for instance, `Queen.from_shapefile(path)` looks something like:

```python
@classmethod
def from_shapefile(cls, path, **kwargs):
    collection = pysal.open(path)
    return cls.from_iterable(collection, **kwargs)

@classmethod
def from_iterable(cls, iterable, **kwargs):
    neighbors, ids = buildContiguity(iterable, **kwargs)
    return cls(neighbors, id_order=ids)
```

I've also got a `driver` flag, but right now, I get different results using rtree and ContiguityWeightsPolygons:

In [10]:
Qrtree = cont.Queen.from_shapefile(path, driver='rtree')

In [11]:
Qrtree.neighbors == Qreference.neighbors

False

I've also been paying attention to reading weights from file (which I believe is the part of FileIO that won't go away with better GDAL/OGR/Fiona/Geopandas packaging)

In [ ]:
w0 = W2.from_file(ps.examples.get_path('columbus.gal'))
w1 = W2.from_file(ps.examples.get_path('wmat.mtx'))
w2 = W2.from_file(ps.examples.get_path('wmat.dat'))

For some reason, the `dat` reads in floats and the `mtx` reads in ints, so straight comparisons don't work yet. 